In [5]:
from Classical_Building_Blocks import knapsack_solver_classical
from Quantum_Building_Blocks import knapsack_solver_quantum

ModuleNotFoundError: No module named 'qiskit'

# KNAPSACK PROBLEM
<!--     As part of the capacitated hetrogenous vehicle routing problem -->

# Table of Contents

1. [Introduction](#Introduction)
1. [Data Loading](#Data-Loading)
1. [Classical Method](#Classical-Method)
   * [Description](#Description)
   * [Results](#Resaults)
1. [Quantum Method](#Quantum-Method)
   * [docplex](#Docplex)
   * [QAOA](#QAOA)
   * [Other Algorithm](#Other-Algorithms)
   * [Results](#Resaults)
   * [Quantum-Circuit](#Quantum-Resaults)
1. [Benchmarking](#Benchmarking)
   * [Quantum Advantages](#Quantum-Advantages)
   * [Algorithms Benchmarking](#Algorithms-benchmarking)
1. [Conclusions](#Conclusions)
1. [Future Work](#Future-Work)

# Introduction
The Knapsack Problem is a classic computational optimization challenge. This problem poses the question: given a list of items, each with a specific weight and value, and a knapsack with a limited carrying capacity, what combination of items should be chosen to maximize the total value without surpassing the knapsack's weight limit?

In the context of this notebook, we will be applying our exploration of the Knapsack Problem to a more complex scenario - the Capacitated Vehicle Routing Problem (CVRP) as discussed in the paper accessible via the following link: https://arxiv.org/pdf/2110.06799.pdf.

The C-HVRP (capacitated hetrogenous vehicle routing problem) involves not just deciding what items to pack in a knapsack (or, in this case, cargo in a vehicle), but also determining the best routes for a fleet of vehicles, each with its own capacity limitation, delivering goods to a set of customers.

To tackle these problems, we will employ a variety of optimization algorithms, including a brute force approach, the Quantum Approximate Optimization Algorithm (QAOA), and several others. These methods will be thoroughly compared and benchmarked, providing an insight into their respective strengths and weaknesses. We aim to highlight the different results and computational efficiencies these algorithms offer when used to solve such complex optimization problems.

The Hamiltonian which we need to simulate and maximized, as presented in the papre, are shown are:


$$ Ha = A (1 - \sum\limits_1^W(Z_n))^2 + A(\sum\limits_1^W(nZ_n) - \sum\limits_1^N(W_iX_i))^2 $$

$$Hb = -B \sum\limits_i^N(c_i x_i)$$



We can extract out of it two expressions: Quadratic & Linear, which we then solve and optimize using various quantum optimization methods

Evantually, when it will be runn together with the HVRP solution, this is the hamiltonian to be parsed, as it containt terms that regards the HVRP problem
see [link to HVRP main page]
$$
Ha = A * \sum\limits_{v}\left(\left( \sum\limits_{k=0}^{M^v-1} (2^k z_k^v + (Q^v + 1 - 2^{M^v}  ) z_M^v\right) -  \sum\limits_{a,i}q_i y_{ia}^v\right)^2
$$



# Data Loading


In [9]:
data_file = 'data/small_data'
max_knapsack_weight = 25

# Classical Method

## Description

Solving the knapsack as in a classical 0/1 Knapsack Problem using Dynamic Programming, specifically a tabulation (or bottom-up) approach is presented here.
The function knapsack_solver_classical(W, wt, v, N) solves the problem by constructing a two-dimensional table K[][] in bottom-up fashion.

In this function, `W` is the maximum weight the knapsack can carry, `wt` is the list of weights for each item, `v` is the list of values for each item, and `N` is the number of items. 
`K[i][w]` represents the maximum value that can be obtained with a knapsack capacity `w` using the first `i` items.
The function iterates over every item and for each one, it considers all possible maximum weights from 0 to `W`. If the weight of the current item is less than or equal to the current capacity `w`, it determines whether to include the item in the optimal subset or not by comparing the total value with or without the current item (`v[j-1] + K[j-1][w-wt[j-1]]` vs `K[j-1][w])`, choosing the maximum of the two. If the weight of the current item is greater than w, it simply carries over the maximum value found for the previous item at the same capacity. The function returns the maximum possible value for the given knapsack capacity and the entire table `K`.

In the next part of the code, the specific items included in the optimal solution are determined by backtracking through the K table, starting from `K[N][W]`. If the value at `K[j][w]` is different from the value at `K[j-1][w]`, it indicates that the jth item was included in the optimal subset. The code keeps track of these chosen items, their corresponding weights, and values, and continuously reduces the remaining capacity of the knapsack by the weight of the chosen item. This process continues until no items are left (`j = 0`) or the knapsack has no remaining capacity (`w = 0`). The final solution is presented in a detailed manner, providing the IDs, values, and weights of the chosen items, along with the total number of items, the total value, and the total weight of the items in the knapsack.

The time complexity of this method `O(N*W)`, where `N` is the number of items and `W`  is the maximum weight of the knapsack. We can solve this problem using an exact solution approach within a reasonable time since the number of combinations is limited, but when the number of items becomes huge, it will be impractical to deal with by using a exact solution approach.

## Resaults

In [10]:
chosen_objects, chosen_values, chosen_weights, max_weigth, total_objects, total_value, chosen_weights_sum = knapsack_solver_classical(data_file, max_knapsack_weight)

print("your knapsack will contain:")
for i in range(len(chosen_objects)):
    print("Object:", chosen_objects[i], "- Value:", chosen_values[i], "- Weight:", chosen_weights[i])
print("The maximum weight the knapsack can have is:", max_weigth, "\nAnd according to the data provided, we can put", total_objects, "objects inside it\nwith a total value of", total_value, "\nand a total weight of", chosen_weights_sum)

your knapsack will contain:
Object: 2 - Value: 6 - Weight: 3
Object: 3 - Value: 8 - Weight: 3
Object: 4 - Value: 7 - Weight: 5
Object: 6 - Value: 4 - Weight: 7
Object: 7 - Value: 6 - Weight: 5
The maximum weight the knapsack can have is: 25 
And according to the data provided, we can put 1 objects inside it
with a total value of 31 
and a total weight of 23


# Quantum Method

## Docplex

Docplex is a powerful optimization modeling and solving library that provides a comprehensive suite of tools for tackling complex optimization problems. Developed by IBM, Docplex offers a user-friendly and intuitive interface, making it accessible to both experts and beginners in the field of optimization. With Docplex, users can formulate their optimization problems using a high-level mathematical modeling language, allowing them to express the constraints and objectives of their problems in a clear and concise manner. The library supports a wide range of mathematical programming techniques, including linear programming, mixed-integer programming, and quadratic programming, among others. Furthermore, Docplex incorporates advanced algorithms and solvers that efficiently explore the solution space and find optimal or near-optimal solutions to the given problems. By leveraging the capabilities of Docplex, users can streamline decision-making processes, improve resource allocation, and optimize various aspects of their operations, ultimately leading to cost savings, increased efficiency, and enhanced performance.

## QAOA

The Quantum Approximate Optimization Algorithm (QAOA) is a quantum computing approach that leverages the power of quantum systems to solve optimization problems. One of the problems it can address is the Knapsack problem, which involves maximizing the value of items that can be packed into a limited capacity bag. QAOA tackles such NP-hard optimization problems by mapping them to a Quadratic Unconstrained Binary Optimization (QUBO) problem, also known as the Ising formulation. In this process, the problem is transformed into a quadratic form, which represents the energy landscape of the system. QAOA then utilizes a sequence of quantum gates to explore this landscape and identify the optimal solution.

To solve an NP-hard optimization problem using the QAOA algorithm, several steps need to be followed. First, the problem must be formulated as a QUBO problem, which involves transforming the constraints and objectives into a quadratic form. This is done by assigning binary variables to the problem's elements and defining a cost function that represents the objective and constraints. Next, the QUBO problem is encoded into a quantum circuit, which is composed of a series of quantum gates that manipulate the qubits' states. The circuit parameters, known as angles, are then optimized to minimize the cost function and obtain the solution with the highest value.

The QAOA algorithm is closely related to IBM's docplex optimization library, which provides a high-level programming interface for formulating and solving optimization problems. By using docplex, users can define the problem's constraints, objective function, and decision variables in a convenient manner. This library can be integrated with QAOA to easily convert the problem into a QUBO form and facilitate the transformation of the optimized angles into a quantum circuit. This combination of QAOA and docplex enables researchers and practitioners to effectively solve complex optimization problems, including the Knapsack problem, leveraging the power of quantum computing and quantum-inspired techniques.

## Other Algorithms 

SPSA (Simultaneous Perturbation Stochastic Approximation) is a powerful optimization algorithm that is commonly used to solve difficult optimization problems. It belongs to the class of derivative-free optimization methods and is particularly well-suited for problems with noisy or black-box objective functions. SPSA iteratively estimates the gradient of the objective function by perturbing the input variables in a random manner and evaluating the corresponding function values. These gradient estimates are then used to update the variables in the direction that minimizes the objective function. SPSA's ability to approximate gradients using noisy evaluations makes it a valuable tool for solving optimization problems in various domains, including the knapsack problem.

COBYLA (Constrained Optimization BY Linear Approximations) is another optimization algorithm commonly used to solve constrained optimization problems. It belongs to the class of derivative-free optimization methods and is particularly effective when dealing with problems where the objective and constraint functions can only be evaluated, but their gradients cannot be computed or are not available. COBYLA approximates the objective function and constraints by using linear models around the current point and iteratively updates the variables to minimize the objective function while satisfying the constraints. This makes COBYLA suitable for solving constrained versions of optimization problems like the knapsack problem.

When it comes to solving NP-hard optimization problems like the knapsack problem using the QAOA (Quantum Approximate Optimization Algorithm) or QUBO (Quadratic Unconstrained Binary Optimization) algorithm, several steps need to be taken. First, the problem needs to be formulated as a QUBO model, which involves mapping the problem variables and constraints into binary variables and quadratic terms. This formulation is related to the quadratic form or Ising formulation commonly used in optimization. Once the problem is in the QUBO form, the QAOA algorithm can be applied to find the optimal or near-optimal solution. The QAOA algorithm uses quantum circuits to evolve a quantum state that represents a candidate solution, and the parameters of the quantum circuits are optimized to maximize the objective function. To solve the QUBO problem and implement the QAOA algorithm, frameworks like docplex can be used. Docplex provides tools for modeling and solving mathematical optimization problems, including QUBO models, making it easier to formulate and solve complex optimization problems using the QAOA algorithm.

## Quantum-Resaults

In [ ]:
knapsack_solver_quantum(data_file, max_knapsack_weight)